In [1]:
import pickle
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [2]:
X = pd.read_feather('../input/amex-imputed-and-1hot-encoded/X_train.parquet')
y = pd.read_feather('../input/amex-imputed-and-1hot-encoded/y_train.parquet')

X = X.set_index('customer_ID')
y = y.set_index('customer_ID')

In [3]:
X = X.iloc[0:200]
y = y.iloc[0:200]

In [4]:
dtc = DecisionTreeClassifier()
dtc = dtc.fit(X, y)

In [5]:
pickle.dump(dtc, open('decision_tree_classifier_model.sav', 'wb'))